### The Gears of Neural Networks: Tensor Operations

#### Element-wise operations

In [30]:
import numpy as np

ReLU formula in math form:

$$
\operatorname{ReLU}(x) = \max(0, x) =
\begin{cases}
0, & x < 0 \\
x, & x \ge 0
\end{cases}
$$

In [31]:
def naive_relu(x):
    assert len(x.shape) == 2
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

In [32]:
x = np.array([
    [-2.0, -1.0, 0.0],
    [1.0, 2.0, -3.0]
])

naive_relu(x)

array([[0., 0., 0.],
       [1., 2., 0.]])

#### What `naive_relu` is doing

The `naive_relu(x)` function applies the ReLU (Rectified Linear Unit) activation element-wise to a 2D NumPy array `x`. It first makes a copy of `x` so the original isn’t modified, then loops over every element and replaces any negative value with `0`, leaving positive values unchanged. The result is a new array where all entries are `>= 0`.

In [33]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

In [34]:
x = np.array([[1.0, -2.0]])
y = np.array([[0.5, -2.0]])

naive_add(x, y)

array([[ 1.5, -4. ]])

#### What `naive_add` is doing

The `naive_add(x, y)` function performs element-wise addition on two 2D NumPy arrays of the same shape. It first checks that both inputs are 2D and have identical shapes, then makes a copy of `x` so the original isn’t modified. It loops over every index `(i, j)` and adds `y[i, j]` to `x[i, j]`, returning the resulting summed array.

#### Numpy implementation

In [35]:
z = x + y 
print("Element-wise Addition:", z)

z = np.maximum(z, 0.0)
print("Element-wise ReLU:", z)

Element-wise Addition: [[ 1.5 -4. ]]
Element-wise ReLU: [[1.5 0. ]]


In [36]:
import time

x = np.random.random((20, 100))
y = np.random.random((20, 100))

t0 = time.time()
for _ in range(1000):
    z = x + y
    z = np.maximum(z, 0.0)
print("Took: {0:.3f}s".format(time.time() - t0))

Took: 0.002s


In [37]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("Took: {0:.3f}s".format(time.time() - t0))

Took: 0.847s


### Broadcasting

#### What broadcasting means

Broadcasting is NumPy’s way of automatically expanding arrays with smaller shapes so they can participate in element-wise operations with larger arrays, without actually copying data. For example, adding an array of shape `(32, 10)` and a vector of shape `(10,)` works because NumPy “stretches” the `(10,)` vector across the 32 rows, effectively treating it like a `(32, 10)` array during the computation.

In [38]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

X = np.random.random((32, 10)) # random matrix with shape (32, 10)
y = np.random.random((10,))    # random vector with shape (10,) 

In [39]:
print("First 5 rows of X matrix:\n", X[:5])
print("X shape:", X.shape)
print("\ny vector:", y)
print("y shape:", y.shape)

First 5 rows of X matrix:
 [[0.79062779 0.65217026 0.20900503 0.01058485 0.86479128 0.32262061 0.22395473 0.57653581 0.47777493 0.38306532]
 [0.52432509 0.44490618 0.19633556 0.86543527 0.97817102 0.17801915 0.76217239 0.11327065 0.52306491 0.39554841]
 [0.98606372 0.75911218 0.45545534 0.29847213 0.35865598 0.8942043  0.76750072 0.93317663 0.65832518 0.35672675]
 [0.4944778  0.29658679 0.93369965 0.07929469 0.77656357 0.42283195 0.43110383 0.2516682  0.48897136 0.79528985]
 [0.05504256 0.34757323 0.04430963 0.63771758 0.41228787 0.6502988  0.24850367 0.60374056 0.34686502 0.03788793]]
X shape: (32, 10)

y vector: [0.42654821 0.63030087 0.19959569 0.16127576 0.92489257 0.97528068 0.70265436 0.31616714 0.4602335  0.74441259]
y shape: (10,)


In [40]:
# Add an empty first axis to y, changing the shape to (1, 10) now
Y = np.expand_dims(y, axis=0) 

In [41]:
print("Y:", Y)
print("Y shape:", Y.shape)

Y: [[0.42654821 0.63030087 0.19959569 0.16127576 0.92489257 0.97528068 0.70265436 0.31616714 0.4602335  0.74441259]]
Y shape: (1, 10)


In [42]:
# Repeat `y` 32 times alongside this new axis, so that we end up with a tensor Y with shape (32, 10), 
# where Y[i, :] == y for i in range(0, 32)
Y = np.tile(y, (32, 1))

In [43]:
print("Y first 5 rows of matrix:\n", Y[:5])
print("Y shape:", Y.shape)

Y first 5 rows of matrix:
 [[0.42654821 0.63030087 0.19959569 0.16127576 0.92489257 0.97528068 0.70265436 0.31616714 0.4602335  0.74441259]
 [0.42654821 0.63030087 0.19959569 0.16127576 0.92489257 0.97528068 0.70265436 0.31616714 0.4602335  0.74441259]
 [0.42654821 0.63030087 0.19959569 0.16127576 0.92489257 0.97528068 0.70265436 0.31616714 0.4602335  0.74441259]
 [0.42654821 0.63030087 0.19959569 0.16127576 0.92489257 0.97528068 0.70265436 0.31616714 0.4602335  0.74441259]
 [0.42654821 0.63030087 0.19959569 0.16127576 0.92489257 0.97528068 0.70265436 0.31616714 0.4602335  0.74441259]]
Y shape: (32, 10)


In [44]:
print("Adding first 5 rows of X and Y:", X[:5] + Y[:5])

Adding first 5 rows of X and Y: [[1.217176   1.28247114 0.40860072 0.17186061 1.78968385 1.2979013  0.92660909 0.89270295 0.93800844 1.1274779 ]
 [0.95087329 1.07520706 0.39593125 1.02671103 1.90306359 1.15329983 1.46482675 0.42943779 0.98329842 1.139961  ]
 [1.41261193 1.38941305 0.65505103 0.45974789 1.28354855 1.86948498 1.47015508 1.24934376 1.11855868 1.10113933]
 [0.921026   0.92688766 1.13329534 0.24057044 1.70145614 1.39811264 1.13375819 0.56783533 0.94920486 1.53970244]
 [0.48159076 0.9778741  0.24390532 0.79899334 1.33718044 1.62557948 0.95115803 0.9199077  0.80709852 0.78230052]]


In [45]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2    # rank-2 NumPy tensor
    assert len(y.shape) == 1    # NumPy vector
    assert x.shape[1] == y.shape[0]
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += [y[j]]
    return x

In [46]:
x = np.random.random((64, 3, 32, 10)) 
y = np.random.random((32, 10))

#### What `x` and `y` represent here

- `x = np.random.random((64, 3, 32, 10))`  
  This is a 4D tensor. You can read the shape as:  
  - 64: number of samples in the batch  
  - 3: channels per sample (e.g., RGB or feature maps)  
  - 32: height (number of rows)  
  - 10: width (number of columns)  

- `y = np.random.random((32, 10))`  
  This is a 2D tensor with the same height and width as each channel of `x` (`32 × 10`). It can be thought of as a single “feature map” that could be broadcast or added to each channel/location in `x`.

In [47]:
z = np.maximum(x, y)

In [48]:
z.shape

(64, 3, 32, 10)

#### Tensor product

The tensor product, also called dot product or matmul (short for “matrix multiplication”) is
one of the most common, most useful tensor operations.
In NumPy, a tensor product is done using the `np.matmul` function, and in Keras, with
the `keras.ops.matmul` function. Its shorthand is the `@` operator in Python

In [49]:
x = np.random.random((32,))
y = np.random.random((32,))

z = np.matmul(x, y) # Dot product
z = x @ y # Equivalent operation

In [50]:
z

np.float64(6.394898403035594)

In mathematical notation, you’d note the operation with a dot (•) (hence the name
“dot product”):

`z = x • y`

In [51]:
def naive_vector_product(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    z = 0.0
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

In [52]:
x = np.array([1.0, 2.0, 3.0])
y = np.array([4.0, 5.0, 6.0])

naive_vector_product(x, y)

np.float64(32.0)

1⋅4 + 2⋅5 + 3⋅6 = 4 + 10 + 18 = 32

In [53]:
def naive_matrix_vector_product(x, y):
    assert len(x.shape) == 2 # x is a NumPy matrix.
    assert len(y.shape) == 1 # y is a NumPy vector.
    assert x.shape[1] == y.shape[0] # The 1st dimension of x must equal the 0th dimension of y!
    z = np.zeros(x.shape[0]) # This operation returns a vector of 0s with as many rows as x.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            print(f"x[i, j]: {x[i, j]}")
            print(f"y[j]: {y[j]}")
            z[i] += x[i, j] * y[j]
            print(f"z: {z} \n")
    return z

In [54]:
x = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0]
])  # shape (2, 3)

y = np.array([7.0, 8.0, 9.0])  # shape (3,)

naive_matrix_vector_product(x, y)

x[i, j]: 1.0
y[j]: 7.0
z: [7. 0.] 

x[i, j]: 2.0
y[j]: 8.0
z: [23.  0.] 

x[i, j]: 3.0
y[j]: 9.0
z: [50.  0.] 

x[i, j]: 4.0
y[j]: 7.0
z: [50. 28.] 

x[i, j]: 5.0
y[j]: 8.0
z: [50. 68.] 

x[i, j]: 6.0
y[j]: 9.0
z: [ 50. 122.] 



array([ 50., 122.])

In [55]:
def naive_matrix_vector_product_v2(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        print(f"x[{i}]:", x[i, :])
        z[i] = naive_vector_product(x[i, :], y)
    return z

In [56]:
x = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0]
])  # shape (2, 3)

y = np.array([7.0, 8.0, 9.0])  # shape (3,)

naive_matrix_vector_product_v2(x, y)

x[0]: [1. 2. 3.]
x[1]: [4. 5. 6.]


array([ 50., 122.])

In [57]:
def naive_matrix_product(x, y):
    assert len(x.shape) == 2 # NumPy Matrix
    assert len(y.shape) == 2 # NumPy Matrix
    assert x.shape[1] == y.shape[0] # The 1st dimension of x must equal the 0th dimension of y
    z = np.zeros((x.shape[0], y.shape[1])) # This operation returns a matrix of 0s with a specific shape.
    for i in range(x.shape[0]): # Iterates over the rows of x
        for j in range(y.shape[1]): # Iterates over the columns of y    
            row_x = x[i, :]
            print("x", row_x)
            column_y = y[:, j]
            print("y", column_y)
            z[i, j] = naive_vector_product(row_x, column_y)
            print("z", z, "\n")
    return z

In [58]:
x = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0]
])  # shape (2, 3)

y = np.array([
    [7.0, 8.0],
    [9.0, 10.0],
    [11.0, 12.0]
])  # shape (3, 2)

naive_matrix_product(x, y)

x [1. 2. 3.]
y [ 7.  9. 11.]
z [[58.  0.]
 [ 0.  0.]] 

x [1. 2. 3.]
y [ 8. 10. 12.]
z [[58. 64.]
 [ 0.  0.]] 

x [4. 5. 6.]
y [ 7.  9. 11.]
z [[ 58.  64.]
 [139.   0.]] 

x [4. 5. 6.]
y [ 8. 10. 12.]
z [[ 58.  64.]
 [139. 154.]] 



array([[ 58.,  64.],
       [139., 154.]])